In [30]:
# Split up the monolithic event.parquet into individual seasons

import pandas as pd
import boxball_loader as bbl

from typing import Sequence

In [ ]:

df = pd.read_parquet('../data/retrosheet/event.parquet')
df.shape

In [9]:
event_gms = df['game_id'].value_counts().index.values
gms = pd.read_parquet('../data/mine/gamelog_enhanced.parquet')[['game_id', 'date', 'game_type']].query('game_id in @event_gms')
gms.shape


(179190, 3)

In [10]:
ev = pd.merge(left=df, right=gms, on='game_id')
ev.shape


(14215796, 162)

In [16]:
ev = bbl.fixup_event_data(ev)
ev['yr'] = ev['date'].dt.year

In [15]:
df.columns

Index(['game_id', 'away_team_id', 'inn_ct', 'bat_home_id', 'outs_ct',
       'balls_ct', 'strikes_ct', 'pitch_seq_tx', 'away_score_ct',
       'home_score_ct',
       ...
       'run2_fate_id', 'run3_fate_id', 'fate_runs_ct', 'ass6_fld_cd',
       'ass7_fld_cd', 'ass8_fld_cd', 'ass9_fld_cd', 'ass10_fld_cd',
       'unknown_out_exc_fl', 'uncertain_play_exc_fl'],
      dtype='object', length=160)

In [19]:
yrs = ev['yr'].value_counts().sort_index().index.values
yrs

array([1903, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914,
       1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925,
       1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936,
       1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947,
       1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958,
       1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969,
       1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980,
       1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [28]:
#yr = 2000
for yr in yrs:
    if yr < 2000:
        print(yr)
        ev_yr = ev.query('yr==@yr')
        ev_yr.to_parquet(f'../data/retrosheet/event_yearly/event_{yr}.parquet')

1903
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999


In [50]:
def load_event_data(seasons: bbl.Seasons, requested_columns: Sequence[str], pa_only=True, game_types=bbl.GameType.RS) -> pd.DataFrame:
    required_cols = ['game_id', 'bat_event_fl', 'h_fl', 'event_cd', 'ab_fl']
    columns = list(set(required_cols+requested_columns))
#    hash_key = (tuple([seasons, tuple(sorted(columns))]))
#    cache_filepath = get_cache_filename('event', hash_key)
    if False: #os.path.isfile(cache_filepath):
        ev = pd.read_parquet(cache_filepath)
    else:
        ev = pd.concat([load_event_data_season(yr, columns) for yr in seasons])
#        ev.to_parquet(cache_filepath)

    ev = ev[ev['bat_event_fl']] if pa_only else ev
    ev = bbl.filter_on_game_types(ev, game_types)
    return ev

In [43]:
def load_event_data_season(season: int, columns: Sequence[str]) -> pd.DataFrame:
    print(season)
    ev_yr = pd.read_parquet(f'../data/retrosheet/event_yearly/event_{season}.parquet')[columns]
    print(ev_yr.shape)
    return ev_yr

In [34]:
required_cols = ['game_id', 'bat_event_fl', 'h_fl', 'event_cd', 'ab_fl']



load_event_data_season(2019, required_cols)

,game_id,bat_event_fl,h_fl,event_cd,ab_fl
6111,ALS201907090,True,0,2,True
6112,ALS201907090,True,0,3,True
6113,ALS201907090,True,0,3,True
6114,ALS201907090,True,1,20,True
6115,ALS201907090,True,0,2,True
...,...,...,...,...,...
12181910,WAS201909290,True,0,3,True
12181911,WAS201909290,True,0,2,True
12181912,WAS201909290,True,0,14,False
12181913,WAS201909290,True,0,2,True


In [51]:
load_event_data(bbl.Seasons(2018, 2020), [])

2018
(193809, 5)
2019
(194946, 5)
2020
(72711, 5)


NameError: name 'cache_filepath' is not defined

In [48]:
bbl.Seasons(2018, 2020)

[2018, 2019, 2020]